# Qubits, Paridad e Ising 

- original_integer
- base_2
- rotate 
- Operador K 
- Diagonalización por bloques ¿?

In [222]:
using LinearAlgebra
using Plots

In [223]:
# Primero definimos algunas funciones auxiliares que nos ayudarán a convertir de un número a una lista de bits y viceversa.
"""
La función convierte un número en binario a su entero original.
"""
function original_integer(list)
    return parse(Int, join(list); base=2)
end


# @doc Recibe un entero y lo escribe en binario, el pad nos dice qué tantos ceros tendrá la representación binaria e.g pad = 2 es 00, pad = 3 es 000

function base_2(integer; pad= nothing)
    if pad == nothing
        return reverse(digits(integer, base = 2))
    else
        return reverse(digits(integer, base = 2, pad = pad))
    end
end

base_2 (generic function with 1 method)

## Hola

In [224]:
A = base_2(5; pad=nothing)
@doc original_integer
# @doc base_2

La función convierte un número en binario a su entero original.


In [225]:
original_integer([1,1,1,1,1])

31

In [226]:
function rotate(index,N)
    vec = base_2(index, pad=N)
    return original_integer([vec[end];vec[1:end-1]]), index
end

rotate (generic function with 1 method)

In [227]:
rotate(1,3)

(4, 1)

In [228]:
m = 3 # Número de Qubits
function OperatorK(m) # m = particles
    am = original_integer(fill(1,m))
    A = zeros(am+1,am+1) #la primer posición es el cero

    for i in 0:am # m es el número de qubits 
       A[ rotate(i,m)[1]+1, i+1 ] = 1
    end
    return A
end
OperatorK(3)

8×8 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [229]:
# Definición de las matrices de Pauli
sigmax = [0 1.0; 1 0]
sigmay = [0 -1.0im; im 0]  # im representa la unidad imaginaria √(-1)
sigmaz = [1.0 0; 0 -1]
sigmas = Dict(1=>sigmax, 2=>sigmay, 3=>sigmaz)
function Sigma(indice, pos, totalparticulas)
    id = I(2) #[1.0 0; 0 1.0]
    mat = sigmas[indice]
    list = []
    for i in 0:totalparticulas-1
        if i == pos 
            append!(list,[mat])
        else 
            append!(list,[id])
        end
        #println("$list")
    end
    return kron(list...)
end

Sigma (generic function with 1 method)

In [230]:
# En un campo externo h 
J = 1
function IsingHomogeneo(N,b; cerrada = false) #Keyword Arguments
    H = 0
    totalparticulas = N
    if cerrada == false 
        M = (N - 1)
    elseif cerrada == true 
        M = N 
    end

    for i in 0:(M-1)
        H = H .+ J*(Sigma(3, mod(i,N),totalparticulas)*Sigma(3,mod((i+1),N),totalparticulas))
    end
    for j in 0:N-1
        H = H .+ b*Sigma(1,j,totalparticulas)
    end
    return H 
end

IsingHomogeneo (generic function with 1 method)

In [231]:
function ProjectionParity(N,b, Bloque; cerrada = false)
    H = IsingHomogeneo(N,b, cerrada = false)
    # P = parity(N)
    OpK = OperatorK(N)
    # vals, v = eigen(P) 
    valsK, vecK = eigen(OpK)
    valsunique = roundedvals(valsK)[1]
    vals = roundedvals(valsK)[2]

    Bloques = Dict()
    for i in 1:length(valsunique)
        Bloques[i] = length(findall(vals .== valsunique[i]))
    end

    vec = vecK[:, findall(vals .== valsunique[Bloque])]

    num = Bloques[Bloque]
    Projection = zeros(ComplexF64,num,num)

    for k in 1:num
        for j in 1:num
            Projection[j,k] = vec[:,j]'*H*vec[:,k]
        end
    end
    return Projection
end

ProjectionParity (generic function with 1 method)

In [232]:
ProjectionParity(4,1,1,cerrada = false)

UndefVarError: UndefVarError: `roundedvals` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [233]:
N = 4
listProj1 = []
for b in 0:0.01:2
     H1 = ProjectionParity(N,b,1,cerrada = false)
     ValsProj1 = eigen(H1).values
     append!(listProj1,map(x->[b,x],ValsProj1))
end
x_listProj = [l[1] for l in listProj1]
y_listProj = [l[2] for l in listProj1]

listProj2 = []
for b in 0:0.01:2
     H2 = ProjectionParity(N,b,2, cerrada = false)
     ValsProj2 = eigen(H2).values
     append!(listProj2,map(x->[b,x],ValsProj2))
end
x_listProj2 = [l[1] for l in listProj2]
y_listProj2 = [l[2] for l in listProj2]

listProj3 = []
for b in 0:0.01:2
     H3 = ProjectionParity(N,b,3, cerrada = false)
     ValsProj3 = eigen(H3).values
     append!(listProj3,map(x->[b,x],ValsProj3))
end
x_listProj3 = [l[1] for l in listProj3]
y_listProj3 = [l[2] for l in listProj3]

listProj4 = []
for b in 0:0.01:2
     H4 = ProjectionParity(N,b,4, cerrada = false)
     ValsProj4 = eigen(H4).values
     append!(listProj4,map(x->[b,x],ValsProj4))
end
x_listProj4 = [l[1] for l in listProj4]
y_listProj4 = [l[2] for l in listProj4]

UndefVarError: UndefVarError: `roundedvals` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [234]:
scatter(real(x_listProj),real(y_listProj), color = "coral", markersize =2.5)
scatter!(real(x_listProj2), real(y_listProj2), color = "blue", markersize = 2.5)
scatter!(real(x_listProj3), real(y_listProj3), color = "red", markersize = 2.5)
scatter!(real(x_listProj4), real(y_listProj4), color = "green", markersize = 2.5)


UndefVarError: UndefVarError: `x_listProj` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# Merge Two Integers

La función recibe tres enteros merge_two_integers(a, b, mask)

Los dígitos binarios de $a$ se colocan en los 1s de mask

In [235]:
mask = original_integer([0,1,1,0,0])
a = original_integer([1,1])
b = original_integer([1,0,1])
mask, a , b
original_integer([1,1,1,0,1])

29

In [236]:
original_integer([1,0,0,0])

8

In [237]:
mask & 1

0

In [238]:
result = 0
bit = b & 1 # ¿El ultimo bit de b es 1 o 0? 
result = result | bit # Esto hace que el último bit de result sea igual al último bit de b
base_2(result, pad = 5)
base_2(mask,pad = 5)


5-element Vector{Int64}:
 0
 1
 1
 0
 0

In [239]:
new_mask = mask >> 1
nm = base_2(new_mask, pad = 4) # Eliminamos un bit de la máscara, el pad es visual
new_b = b >> 1
bm = base_2(new_b, pad = 2) # Eliminamos un bit de b, el pad es visual
new_mask & 1 # El último bit también es 0
bit = new_b & 1 

nm, bm, bit 

([0, 1, 1, 0], [1, 0], 0)

In [240]:
function merge_two_integers(a,b,mask)
   
end

merge_two_integers (generic function with 2 methods)

In [241]:
mask = original_integer([0,1,1,0,0])
a = original_integer([1,0])
b = original_integer([1,0,1])

mask & 1 # El primer índice no tiene un bit, por lo cual no se modifica aún
new_mask = mask >> 1
new_mask & 1 # El segundo índice tampoco tiene un bit, por lo cual no se modifica aún
new_mask2 = new_mask >> 1
new_mask2 & 1 # El tercer índice tiene un bit, por lo cual se modifica
bit = a & 1 # Es el elemento a reemplazar en new_mask2
new_mask3 = new_mask2 |bit
new_mask4 = new_mask3 >> 1 
new_mask4 & 1   # El cuarto índice tiene un bit, por lo cual se modifica

new_mask5 = new_mask4 | bit # Se reemplaza el bit
new_mask6 = new_mask5 >> 1
new_mask6 & 1
new_mask6 == 0 # La máscara ya no tiene más bits y se termina el proceso de target, ahora falta hacer el proceso de untouched 

function merge_two_integers(a,b,mask)
    bit = a & 1 # Es el valor que vamos a reemplazar (Recordar que una vez sea reemplazado, hay que irnos al siguiente dígito)
    k = 0
    while mask & 1 == 0
        new_mask = mask >> 1
        k += 1
    end
    resultados = []

    new_mask = mask

    k2 = 0
    while (new_mask & 1 ==1) & (a & 1 == 1) | k2 == 10
        if a & 1 == 0
          new_mask = new_mask & ~a
        elseif a & 1 == 1
            new_mask = new_mask | a
        end
    push!(resultados, new_mask)
    new_mask = new_mask >> 1
    a = a >> 1
    k2 += 1
    end


    # a >> 1 # Nos movemos al siguiente bit
        #new_mask = new_mask >> 1 
        #a = a >> 1

    

    return base_2(new_mask), base_2(mask),resultados

end

merge_two_integers (generic function with 2 methods)

In [242]:
#merge_two_integers(a,b,mask)

In [243]:
new_mask = original_integer([1,1])

3

In [244]:
mask = original_integer([1,0,0,0])
#new_mask = mask >> 5
bit = 1 &  1
bit << 2
new_mask = mask | bit<<2 #[1,1,0,0] 
new_mask2 = new_mask | bit << 2 # ¿Cómo elimino el bit que ya puse?
new_mask3 = new_mask2 & ~bit << 2

8

In [245]:
base_2(8)

4-element Vector{Int64}:
 1
 0
 0
 0

In [246]:
mask = original_integer([0,1,1,0,0])
a = original_integer([1,0])
b = original_integer([1,0,1])

# El resultado deseado es [1,1,0,0,1]

bit = a & 1
bit2 = a >> 2 & 1 # El segundo bit de a 

# Hay que analizar la máscara 

mask >> 4 & 1 # Podemos colocar un for i in 1:5 para ver si hay bits en la máscara mask >> i & 1

if mask & 1 == 1
    mask = mask | bit 
end
    

In [247]:
# Cómo implementar un 0 si el target tiene 0s 
mask = original_integer([0,1,1,0,1])
a = original_integer([0,0])
bit = a & 1 
bit2 = a >> 1 & 1
if bit == 0 && (mask & 1 == 1) 
    newmask = mask & ~1
end

if bit2 == 0 && (newmask >> 1 & 1 == 1)
    newmask2 = newmask >> 1 & ~1
else 
    newmask2 = newmask
end


12

In [248]:
# Cambiar los 1's de la máscara por 0's del target. Ejemplo 
mask = original_integer([1,1,0,1,0])
a = original_integer([0,0])

index = zeros(length(base_2(mask)))
for i in 1:length(base_2(mask))
    if mask >> (i-1) & 1 == 1
        index[i] = i
    else 
        index[i] = 0
    end
end
idx = index[findall(!iszero, index)]

# Cambiar 1's de mask por 0's del target
for i in 0:length(base_2(a))-1
    if  bit >> i & 1 == 0
        newmask = mask >> (Int(idx[i+1])-1) & ~1 
    end
end
mask, newmask, idx

(26, 12, [2.0, 4.0, 5.0])

In [249]:
mask

newmask2 = mask >> 3 & ~1
base_2(2)

2-element Vector{Int64}:
 1
 0

In [250]:
# Obtener un vector que me de el índice donde se tiene que hacer el cambio
index = zeros(length(base_2(mask)))
mask = original_integer([1,1,0,1,1]) # Me gustaría tener un vector que diga (0,1,3,4) 
for i in 1:length(base_2(mask))
    if mask >> (i-1) & 1 == 1
        index[i] = i
    else 
        index[i] = 0
    end
end
index

5-element Vector{Float64}:
 1.0
 2.0
 0.0
 4.0
 5.0

In [251]:
z = original_integer([0,1,0])
bit = z >> 0 & 1

0

In [252]:
function merge_two_integers(a ,b ,mask , n) #a es el target y b es el untouched
    result = 0
    i=0
    bit_mask= base_2(mask, pad=n)
    while i<=n-1 
        if bit_mask[end-i] == 1
            bit = a & 1
            result = result | bit<<i
            a = a>>1
        else
            bit = b & 1
            result = result | bit<<i
            b = b>>1
        end
        i+=1
    end

    return result
               
end

merge_two_integers (generic function with 2 methods)

In [253]:
mask = original_integer([0,1,0,1,0])
a = original_integer([1,0])
b = original_integer([1,0,1])
A = merge_two_integers(mask,a,b,5)

12

In [254]:
base_2(A, pad = 5)

5-element Vector{Int64}:
 0
 1
 1
 0
 0

In [264]:
function apply_unitary(state, U, target, n)
    new_state = zeros(2^n)
    dimtarget = 2
    dim_untouched = length(state)/2
    for unt in 0: (dim_untouched-1) 
        pos = [merge_two_integers(i, target, mask, n) for i in 0: dimtarget -1] #merge_two_integers(target,untouched,mask,Particles)
    end
    new_state[pos] = U*state[pos]
    return new_state
end

apply_unitary (generic function with 1 method)

In [266]:
pos = [merge_two_integers(i,b,m,5)+1 for i in 0:2-1]

2-element Vector{Int64}:
 21
 22

In [265]:
state = [1,0,0,0,0]
sigmaz
apply_unitary(state, sigmaz, 1, 3)

BoundsError: BoundsError: attempt to access 5-element Vector{Int64} at index [[9, 10]]